In [88]:
import numpy as np
import cvxpy as cp

In [89]:
# HELPER FUNCTIONS
# generate_data is modified version of https://github.com/cvxgrp/cvxbook_additional_exercises/blob/main/python/rob_logistic_reg_data.py
np.random.seed(0x364a_23F5)
d = 40
n = 60
def generate_data(eps):
    
    epsilon = eps

    true_theta = np.random.randn(d)
    true_X = np.random.randn(n, d)
    noise = 2 * epsilon * np.random.rand(n, d) - epsilon

    X = true_X + noise
    y = np.sign(true_X @ true_theta + 0.1 * np.random.rand(n) - 0.05)

    true_X_test = np.random.randn(n, d)
    noise = 2 * epsilon * np.random.rand(n, d) - epsilon

    X_test = true_X_test + noise
    y_test = np.sign(true_X_test @ true_theta + 0.1 * np.random.rand(n) - 0.05)
    return (X, y, X_test, y_test)

def test_classifier(data_matrix, classifications, params):
    num_correct = 0
    for i in range(data_matrix.shape[0]):
        y_hat = cp.sign(params @ data_matrix[i, :]).value
        if y_hat == classifications[i]:
            num_correct += 1
    return num_correct

In [90]:
# train a typical logistic classifier with eps = 1/2
epsilon=0.5
X, y, X_test, y_test = generate_data(0.5)

theta = cp.Variable(d)
lse = cp.logistic

loss = cp.sum( [ lse( -y[i] * theta@X[i, :] ) for i in range(n)] )
prob = cp.Problem(cp.Minimize(loss), [])
prob.solve(solver=cp.CLARABEL)

1.68719193727882e-09

In [91]:
# evaluate
train_correct = test_classifier(X, y, theta.value)
print(f"The non robust logistic model correctly classified {train_correct} data points with eps=0.5 of the training dataset")
train_correct = test_classifier(X_test, y_test, theta.value)
print(f"The non robust logistic model correctly classified {train_correct} data points with eps=0.5 of the testing dataset")

The non robust logistic model correctly classified 60 data points with eps=0.5 of the training dataset
The non robust logistic model correctly classified 36 data points with eps=0.5 of the testing dataset


In [92]:
def train_robust_classifier(eps, norm, X, y):
    """
    bad practice, but norm should either be 1 or 2 (ints) or 'inf' (string)
    """
    theta_robust = cp.Variable(d)
    u = cp.Variable(n)

    lse = cp.logistic

    loss = cp.sum( [ lse( u[i] ) for i in range(n)] )

    constrs = [ -y[i]*theta_robust@X[i, :] + cp.norm((-eps*y[i]*theta_robust), norm) <= u[i]
            for i in range(n)]
    # Note the following doesn't work because y_i can equal -1, and then the constraint is not DCP compliant
    # constrs = [ -y[i]*theta_robust@X[i, :] + (-epsilon*y[i])*cp.norm(theta_robust, 1) <= u[i]
    #            for i in range(n)]

    prob = cp.Problem(cp.Minimize(loss), constrs)
    prob.solve(solver=cp.CLARABEL)
    return theta_robust.value

In [93]:
theta_robust = train_robust_classifier(epsilon, 1, X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_inf uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_inf uncertainty set, eps={epsilon} of the testing dataset")

The robust logistic model correctly classified 47 data points with ell_inf uncertainty set, eps=0.5 of the training dataset
The robust logistic model correctly classified 38 data points with ell_inf uncertainty set, eps=0.5 of the testing dataset


In [94]:
# evaluate
theta_robust = train_robust_classifier(epsilon, 2, X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_2 uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_2 uncertainty set, eps={epsilon} of the testing dataset")

The robust logistic model correctly classified 60 data points with ell_2 uncertainty set, eps=0.5 of the training dataset
The robust logistic model correctly classified 49 data points with ell_2 uncertainty set, eps=0.5 of the testing dataset


In [95]:
theta_robust = train_robust_classifier(epsilon, 'inf', X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logist model correctly classified {train_correct} data points with ell_1 uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logist model correctly classified {train_correct} data points with ell_1 uncertainty set, eps={epsilon} of the testing dataset")

The robust logist model correctly classified 60 data points with ell_1 uncertainty set, eps=0.5 of the training dataset
The robust logist model correctly classified 47 data points with ell_1 uncertainty set, eps=0.5 of the testing dataset


In [98]:
# train a typical logistic classifier with eps = 0.75
epsilon=0.75
X, y, X_test, y_test = generate_data(epsilon)

theta = cp.Variable(d)
lse = cp.logistic

loss = cp.sum( [ lse( -y[i] * theta@X[i, :] ) for i in range(n)] )
prob = cp.Problem(cp.Minimize(loss), [])
prob.solve(solver=cp.CLARABEL)
theta=theta.value

In [99]:
# evaluate
train_correct = test_classifier(X, y, theta)
print(f"The non robust logistic model correctly classified {train_correct} data points with eps=0.75 of the training dataset")
train_correct = test_classifier(X_test, y_test, theta)
print(f"The non robust logistic model correctly classified {train_correct} data points with eps=0.75 of the testing dataset")

The non robust logistic model correctly classified 60 data points with eps=0.75 of the training dataset
The non robust logistic model correctly classified 39 data points with eps=0.75 of the testing dataset


In [100]:
theta_robust = train_robust_classifier(epsilon, 1, X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_inf uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_inf uncertainty set, eps={epsilon} of the testing dataset")

The robust logistic model correctly classified 53 data points with ell_inf uncertainty set, eps=0.75 of the training dataset
The robust logistic model correctly classified 46 data points with ell_inf uncertainty set, eps=0.75 of the testing dataset


In [101]:
# evaluate
theta_robust = train_robust_classifier(epsilon, 2, X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_2 uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_2 uncertainty set, eps={epsilon} of the testing dataset")

The robust logistic model correctly classified 58 data points with ell_2 uncertainty set, eps=0.75 of the training dataset
The robust logistic model correctly classified 47 data points with ell_2 uncertainty set, eps=0.75 of the testing dataset


In [102]:
theta_robust = train_robust_classifier(epsilon, 'inf', X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logist model correctly classified {train_correct} data points with ell_1 uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logist model correctly classified {train_correct} data points with ell_1 uncertainty set, eps={epsilon} of the testing dataset")

The robust logist model correctly classified 60 data points with ell_1 uncertainty set, eps=0.75 of the training dataset
The robust logist model correctly classified 44 data points with ell_1 uncertainty set, eps=0.75 of the testing dataset


In [105]:
# train a typical logistic classifier with eps = 1/2
epsilon = 1
X, y, X_test, y_test = generate_data(epsilon)

theta = cp.Variable(d)
lse = cp.logistic

loss = cp.sum( [ lse( -y[i] * theta@X[i, :] ) for i in range(n)] )
prob = cp.Problem(cp.Minimize(loss), [])
prob.solve(solver=cp.CLARABEL)

2.366440584883235e-09

In [106]:
# evaluate
train_correct = test_classifier(X, y, theta)
print(f"The non robust logist model correctly classified {train_correct} data points with eps=1 of the training dataset")
train_correct = test_classifier(X_test, y_test, theta)
print(f"The non robust logist model correctly classified {train_correct} data points with eps=1 of the testing dataset")

The non robust logist model correctly classified 60 data points with eps=1 of the training dataset
The non robust logist model correctly classified 35 data points with eps=1 of the testing dataset


In [107]:
theta_robust = train_robust_classifier(epsilon, 1, X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_inf uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_inf uncertainty set, eps={epsilon} of the testing dataset")

The robust logistic model correctly classified 50 data points with ell_inf uncertainty set, eps=1 of the training dataset
The robust logistic model correctly classified 41 data points with ell_inf uncertainty set, eps=1 of the testing dataset


In [111]:
# evaluate
theta_robust = train_robust_classifier(epsilon, 2, X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_2 uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logistic model correctly classified {train_correct} data points with ell_2 uncertainty set, eps={epsilon} of the testing dataset")

The robust logistic model correctly classified 54 data points with ell_2 uncertainty set, eps=1 of the training dataset
The robust logistic model correctly classified 41 data points with ell_2 uncertainty set, eps=1 of the testing dataset


In [114]:
theta_robust = train_robust_classifier(epsilon, 'inf', X, y)
train_correct = test_classifier(X, y, theta_robust)
print(f"The robust logist model correctly classified {train_correct} data points with ell_1 uncertainty set, eps={epsilon} of the training dataset")
train_correct = test_classifier(X_test, y_test, theta_robust)
print(f"The robust logist model correctly classified {train_correct} data points with ell_1 uncertainty set, eps={epsilon} of the testing dataset")

The robust logist model correctly classified 60 data points with ell_1 uncertainty set, eps=1 of the training dataset
The robust logist model correctly classified 35 data points with ell_1 uncertainty set, eps=1 of the testing dataset
